In [ ]:
import os
from PIL import Image
import numpy as np

def get_color_percentage(image_path, target_colors):
    """
    Calculate the total percentage of pixels matching any of the target colors
    
    Args:
        image_path (str): Path to the image file
        target_colors (list): List of hex color strings to match ('#RRGGBB')
        
    Returns:
        float: Percentage of pixels matching any target color (0-100)
    """
    try:
        img = Image.open(image_path)
        if img.mode != 'RGB':
            img = img.convert('RGB')
            
        img_array = np.array(img)
        pixels = img_array.reshape(-1, 3)
        
        # Convert target hex colors to RGB tuples
        target_rgb = []
        for hex_color in target_colors:
            hex_color = hex_color.lstrip('#')
            rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
            target_rgb.append(rgb)
        
        # Count matching pixels
        matching_pixels = 0
        for pixel in pixels:
            if tuple(pixel) in target_rgb:
                matching_pixels += 1
                
        total_pixels = len(pixels)
        if total_pixels == 0: # Avoid division by zero for empty images
            return 0
        return (matching_pixels / total_pixels) * 100
        
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return 0

# --- Define Rules with Descriptive Names ---
# We'll change the structure slightly to include a 'name' for easier reporting
deletion_rules_with_names = [
    {'name': 'Green Area (>95%)', 'colors': ['#70E238', '#23C723'], 'threshold': 95},
    {'name': 'Forest Area (>80%)', 'colors': ['#3ECF60'], 'threshold': 80},
    {'name': 'Water Area (>95%)', 'colors': ['#5CEBEB'], 'threshold': 95},
    {'name': 'Farmland (>95%)', 'colors': ['#DFED5B'], 'threshold': 95},
    {'name': 'Other Green Area (>95%)', 'colors': ['#65CB32'], 'threshold': 95},
    # Add more rules here if needed, following the same dictionary structure
]

# --- Script to analyze images in a directory ---
directory_to_scan = r"" # Use raw string for Windows paths

total_images_processed = 0
# Initialize a dictionary to store counts for each rule
rule_match_counts = {rule['name']: 0 for rule in deletion_rules_with_names}

valid_image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')

if not os.path.isdir(directory_to_scan):
    print(f"Error: Directory not found: {directory_to_scan}")
else:
    print(f"Scanning directory: {directory_to_scan}\n")
    for filename in os.listdir(directory_to_scan):
        if filename.lower().endswith(valid_image_extensions):
            image_path = os.path.join(directory_to_scan, filename)
            total_images_processed += 1
            # print(f"Processing {filename}...") # Uncomment for verbose output

            # Check this image against each rule independently
            for rule in deletion_rules_with_names:
                rule_name = rule['name']
                rule_colors = rule['colors']
                rule_threshold_percentage = rule['threshold']

                calculated_percentage = get_color_percentage(image_path, rule_colors)
                
                # If the image meets this specific rule's criteria, increment its count
                if calculated_percentage > rule_threshold_percentage:
                    rule_match_counts[rule_name] += 1
                    # Note: An image might satisfy multiple rules, and it will be counted for each.
                    # If an image should only be counted once even if it meets multiple rules,
                    # the logic would need to be adjusted (e.g., using a set to track processed images per rule category).
                    # For now, this counts an image for every rule it satisfies.
                    # If an image is "Green Area" AND "Forest Area", it increments both.

    # --- Output Results ---
    print("\n--- Analysis Complete ---")
    print(f"Total image files found and processed: {total_images_processed}")
    
    if total_images_processed > 0:
        print("\nBreakdown by rule criteria:")
        for rule_name, count in rule_match_counts.items():
            percentage_of_total = (count / total_images_processed) * 100
            print(f"- {rule_name}: {count} images ({percentage_of_total:.2f}% of total processed images)")
    else:
        print("No image files were processed.")
